In [3]:
from vmdpy import VMD
import numpy as np
import random
from math import log
from scipy.fftpack import hilbert,fft,ifft
import pandas as pd
import time

def GWO(objf, lb, ub, dim, SearchAgents_no, Max_iter):
    # 初始化 alpha, beta, and delta_pos
    Alpha_pos = np.zeros(dim)  # 位置.形成2的列表
    Alpha_score = float("inf")  # 这个是表示“正负无穷”,所有数都比 +inf 小；正无穷：float("inf"); 负无穷：float("-inf")

    Beta_pos = np.zeros(dim)
    Beta_score = float("inf")

    Delta_pos = np.zeros(dim)
    Delta_score = float("inf")  # float() 函数用于将整数和字符串转换成浮点数。

   

    # 初始化所有狼的位置
    Positions = np.zeros((SearchAgents_no, dim))
    for i in range(dim):  
        Positions[:, i] = np.ones(SearchAgents_no)*ub[i]  
    Convergence_curve = np.zeros(Max_iter)

    #迭代寻优
    for l in range(0, Max_iter):  # 迭代1000
        for i in range(0, SearchAgents_no):  
            # 返回超出搜索空间边界的搜索代理

            for j in range(dim):  # 30
                Positions[i, j] = np.clip(Positions[i, j], lb[j], ub[
                    j])  # clip这个函数将将数组中的元素限制在a_min, a_max之间，大于a_max的就使得它等于 a_max，小于a_min,的就使得它等于a_min。

            # 计算每个搜索代理的目标函数
            fitness = objf(Positions[i, :])  # 把某行数据带入函数计算
            # print("经过计算得到：",fitness)

            
            if fitness < Alpha_score:
                Alpha_score = fitness 
                Alpha_pos = Positions[i, :].copy()

            if (fitness > Alpha_score and fitness < Beta_score):
                Beta_score = fitness 
                Beta_pos = Positions[i, :].copy()

            if (fitness > Alpha_score and fitness > Beta_score and fitness < Delta_score):
                Delta_score = fitness  
                Delta_pos = Positions[i, :].copy()
                
       
        print(int(Alpha_pos[0]),Alpha_pos[1])
        # print(int(Beta_pos[0]),Beta_pos[1])
        # print(int(Delta_pos[0]),Delta_pos[1])
        print(Alpha_score)
        # print(Beta_score)
        # print(Delta_score)
        a = 2 - l * ((2) / Max_iter);  #   a从2线性减少到0

        for i in range(0, SearchAgents_no):
            for j in range(0, dim):
                r1 = random.random()  # r1 is a random number in [0,1]主要生成一个0-1的随机浮点数。
                r2 = random.random()  # r2 is a random number in [0,1]

                A1 = 2 * a * r1 - a  # Equation (3.3)
                C1 = 2 * r2  # Equation (3.4)
                # D_alpha表示候选狼与Alpha狼的距离
                D_alpha = abs(C1 * Alpha_pos[j] - Positions[
                    i, j])  # abs() 函数返回数字的绝对值。Alpha_pos[j]表示Alpha位置，Positions[i,j])候选灰狼所在位置
                X1 = Alpha_pos[j] - A1 * D_alpha  # X1表示根据alpha得出的下一代灰狼位置向量

                r1 = random.random()
                r2 = random.random()

                A2 = 2 * a * r1 - a  #
                C2 = 2 * r2;

                D_beta = abs(C2 * Beta_pos[j] - Positions[i, j])
                X2 = Beta_pos[j] - A2 * D_beta

                r1 = random.random()
                r2 = random.random()

                A3 = 2 * a * r1 - a
                C3 = 2 * r2

                D_delta = abs(C3 * Delta_pos[j] - Positions[i, j])
                X3 = Delta_pos[j] - A3 * D_delta

                Positions[i, j] = (X1 + X2 + X3) / 3  # 候选狼的位置更新为根据Alpha、Beta、Delta得出的下一代灰狼地址。

        Convergence_curve[l] = Alpha_score

        if (l % 1 == 0):
            print(['迭代次数为' + str(l) + ' 的迭代结果' + str(Alpha_score)])  # 每一次的迭代结果


df = pd.read_csv('solar2019.csv',header=0, parse_dates=[0],index_col=0)# 读取CSV文件，并设置第一列为日期，作为索引
# print(df.head(10))
x = df.iloc[0:2000,-1].values
print(x)


tau = 0.  # noise-tolerance (no strict fidelity enforcement)
DC = 0  # no DC part imposed
init = 1  # initialize omegas uniformly
tol = 1e-7



#函数
def F1(position):
    K = int(position[0])
    alpha = position[1]
    u, u_hat, omega = VMD(x, alpha, tau, K, DC, init, tol)
    EP = []
    for i in range(K):
        H = np.abs(hilbert(u[i,:]))
        e1 = []
        for j in range(len(H)):
            p = H[j]/np.sum(H)
            e = -p*log(p,2)
            e1.append(e)
        E = np.sum(e1)  
        EP.append(E)
    s = np.sum(EP)/K
    return s

#主程序
func_details = ['F1', [3,100], [10,4000], 2]
function_name = func_details[0]
Max_iter = 40  # 迭代次数
lb = [3,100]  #下界
ub = [10,4000]  #上届
dim = 2   #狼的寻值范围
SearchAgents_no = 20 #寻值的狼的数量
start_time = time.time()  # 记录开始时间
x = GWO(F1, lb, ub, dim, SearchAgents_no, Max_iter)
end_time = time.time()  # 记录结束时间
execution_time = end_time - start_time  # 计算代码执行时间
print("代码执行时间：", execution_time, "秒")

[0. 0. 0. ... 0. 0. 0.]
10 4000.0
10.202274085561708
['迭代次数为0 的迭代结果10.202274085561708']
5 100.0
9.881598277885232
['迭代次数为1 的迭代结果9.881598277885232']
5 100.0
9.881598277885232
['迭代次数为2 的迭代结果9.881598277885232']
5 100.0
9.881598277885232
['迭代次数为3 的迭代结果9.881598277885232']
5 100.0
9.881598277885232
['迭代次数为4 的迭代结果9.881598277885232']
5 100.0
9.881598277885232
['迭代次数为5 的迭代结果9.881598277885232']
5 100.0
9.881598277885232
['迭代次数为6 的迭代结果9.881598277885232']
5 154.24910901952023
9.860459026438715
['迭代次数为7 的迭代结果9.860459026438715']
5 154.24910901952023
9.860459026438715
['迭代次数为8 的迭代结果9.860459026438715']
5 161.69805432147928
9.858997344154993
['迭代次数为9 的迭代结果9.858997344154993']
5 161.69805432147928
9.858997344154993
['迭代次数为10 的迭代结果9.858997344154993']
5 161.69805432147928
9.858997344154993
['迭代次数为11 的迭代结果9.858997344154993']
5 161.69805432147928
9.858997344154993
['迭代次数为12 的迭代结果9.858997344154993']
5 167.5185535474841
9.858596514045004
['迭代次数为13 的迭代结果9.858596514045004']
5 167.5185535474841
9.858596514045004


In [ ]:
代码执行时间： 691.9200639724731 秒


"""
10.202274085561708
['迭代次数为0 的迭代结果10.202274085561708']
10 100.0
9.932501520378349
['迭代次数为1 的迭代结果9.932501520378349']
5 100.0
9.881598277885232
['迭代次数为2 的迭代结果9.881598277885232']
5 100.0
9.881598277885232
['迭代次数为3 的迭代结果9.881598277885232']
5 100.0
9.881598277885232
['迭代次数为4 的迭代结果9.881598277885232']
5 125.95267370179555
9.869915027479902
['迭代次数为5 的迭代结果9.869915027479902']
5 154.66600285364925
9.860356316055185
['迭代次数为6 的迭代结果9.860356316055185']
5 154.66600285364925
9.860356316055185
['迭代次数为7 的迭代结果9.860356316055185']
5 154.66600285364925
9.860356316055185
['迭代次数为8 的迭代结果9.860356316055185']
5 169.18383219674772
9.85865446788217
['迭代次数为9 的迭代结果9.85865446788217']
5 168.72095470231818
9.858645325300916
['迭代次数为10 的迭代结果9.858645325300916']
5 168.72095470231818
9.858645325300916
['迭代次数为11 的迭代结果9.858645325300916']
5 168.72095470231818
9.858645325300916
['迭代次数为12 的迭代结果9.858645325300916']
5 168.72095470231818
9.858645325300916
['迭代次数为13 的迭代结果9.858645325300916']
5 168.72095470231818
9.858645325300916
['迭代次数为14 的迭代结果9.858645325300916']
5 164.7510396791183
9.858563646719357
['迭代次数为15 的迭代结果9.858563646719357']
5 164.7510396791183
9.858563646719357
['迭代次数为16 的迭代结果9.858563646719357']
5 164.7510396791183
9.858563646719357
['迭代次数为17 的迭代结果9.858563646719357']
5 164.7510396791183
9.858563646719357
['迭代次数为18 的迭代结果9.858563646719357']
5 164.7510396791183
9.858563646719357
['迭代次数为19 的迭代结果9.858563646719357']
5 164.7510396791183
9.858563646719357
['迭代次数为20 的迭代结果9.858563646719357']
5 164.7510396791183
9.858563646719357
['迭代次数为21 的迭代结果9.858563646719357']
5 164.7510396791183
9.858563646719357
['迭代次数为22 的迭代结果9.858563646719357']
5 164.7510396791183
9.858563646719357
['迭代次数为23 的迭代结果9.858563646719357']
5 164.7510396791183
9.858563646719357
['迭代次数为24 的迭代结果9.858563646719357']
5 166.560026017745
9.858528406055418
['迭代次数为25 的迭代结果9.858528406055418']
5 166.560026017745
9.858528406055418
['迭代次数为26 的迭代结果9.858528406055418']
5 166.560026017745
9.858528406055418
['迭代次数为27 的迭代结果9.858528406055418']
5 165.37176807652398
9.85852620542114
['迭代次数为28 的迭代结果9.85852620542114']
5 165.37176807652398
9.85852620542114
['迭代次数为29 的迭代结果9.85852620542114']
5 165.37176807652398
9.85852620542114
['迭代次数为30 的迭代结果9.85852620542114']
5 166.4383257224212
9.858520501987016
['迭代次数为31 的迭代结果9.858520501987016']
5 166.4383257224212
9.858520501987016
['迭代次数为32 的迭代结果9.858520501987016']
5 165.6589285115291
9.858508556929355
['迭代次数为33 的迭代结果9.858508556929355']
5 165.6589285115291
9.858508556929355
['迭代次数为34 的迭代结果9.858508556929355']
5 166.17428533676022
9.858508231679659
['迭代次数为35 的迭代结果9.858508231679659']
5 166.17428533676022
9.858508231679659
['迭代次数为36 的迭代结果9.858508231679659']
5 165.97877171689362
9.858498055790488
['迭代次数为37 的迭代结果9.858498055790488']
5 165.97877171689362
9.858498055790488
['迭代次数为38 的迭代结果9.858498055790488']
5 165.97877171689362
9.858498055790488
['迭代次数为39 的迭代结果9.858498055790488']

"""